<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
import os
from tqdm.notebook import tqdm
import json
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Layer, Dense, Flatten, Dropout, Concatenate, Add, Dot, Multiply, Reshape, Activation, BatchNormalization, SimpleRNNCell, RNN, SimpleRNN, LSTM, Embedding, Bidirectional, TimeDistributed, Conv1D, Conv2D, MaxPool1D, MaxPool2D, GlobalMaxPool1D, GlobalMaxPool2D, AveragePooling1D, AveragePooling2D, GlobalAveragePooling1D, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, CosineSimilarity
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import linear, sigmoid, relu
from tensorflow.keras.initializers import RandomNormal, glorot_uniform, he_uniform, Constant
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import imdb

tqdm.pandas()
plt.style.use("dark_background")

C:\Users\82104\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [82]:
vocab_size = 10000
(tr_X, tr_y), (te_X, te_y) = imdb.load_data(num_words=vocab_size)

In [83]:
lens = sorted([len(doc) for doc in tr_X])
ratio = 0.99
max_len = int(np.quantile(lens, 0.99))
print(f"길이가 가장 긴 문장의 길이는 {np.max(lens)}이고 길이가 {max_len} 이하인 문장이 전체의 {ratio:.0%}를 차지합니다.")

# max_len = 500
tr_X = pad_sequences(tr_X, padding="post", maxlen=max_len)
te_X = pad_sequences(te_X, padding="post", maxlen=max_len)

길이가 가장 긴 문장의 길이는 2494이고 길이가 926 이하인 문장이 전체의 99%를 차지합니다.


In [78]:
inputs = Input(shape=(max_len,))

z = Embedding(input_length=max_len, input_dim=vocab_size, output_dim=128, mask_zero=True)(inputs)
z = Bidirectional(LSTM(units=64, dropout=0.5, return_sequences=True, return_state=False))(z)
z, for_h_state, for_c_state, back_h_state, back_c_state = Bidirectional(LSTM(units=64, dropout=0.5, return_sequences=True, return_state=True))(z)
h_state = Concatenate()([for_h_state, back_h_state])

# Bahdanau attention
h_state = h_state[:, None, :]
unists = 64
attention_scores = Dense(units=1)(tf.nn.tanh(Dense(units=units)(h_state) + Dense(units=units)(z)))
attention_weights = tf.nn.softmax(attention_scores, axis=-1)
context_vec = tf.math.reduce_sum(attention_weights*z, axis=1)

# context_vec, _ = BahdanauAttention(units=64)(z, h_state)

z = Dense(units=20, activation="relu")(context_vec)
z = Dropout(rate=0.5)(z)

outputs = Dense(units=1, activation="sigmoid")(z)
  
model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 926)]        0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, 926, 128)     1280000     ['input_7[0][0]']                
                                                                                                  
 bidirectional_12 (Bidirectiona  (None, 926, 128)    98816       ['embedding_6[0][0]']            
 l)                                                                                               
                                                                                                  
 bidirectional_13 (Bidirectiona  [(None, 926, 128),  98816       ['bidirectional_12[0][0]'] 

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

es = EarlyStopping(monitor="val_loss", mode="auto", verbose=1, patience=1)
model_path = "imdb_bilstm_bahdanau.h5"
mc = ModelCheckpoint(filepath=model_path, monitor="val_acc", mode="auto", verbose=1, save_best_only=True)
hist = model.fit(x=tr_X, y=tr_y, epochs=8, batch_size=256, validation_split=0.2, verbose=2, shuffle=True, callbacks=[es, mc])

Epoch 1/8


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(te_X, te_y)[1]))

25000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================